In [185]:
import pickle

In [186]:
x= np.zeros((1,3))
with open('moderntdv_1000.xml') as f:
    for line in f:
        if line.find('<w ')>-1:
            tags = parse(line)
            x = np.vstack([x,tags])
        
with open('pos_tagged.pickle', 'wb') as f:
    pickle.dump(x,f)
 #       str.find(str, beg=0, end=len(string))

In [184]:
x

array([['0.0', '0.0', '0.0'],
       ['(', 'PAD', '|'],
       ['Jfr', 'VB', '|jämföra..1|'],
       ..., 
       ['arbetet', 'NN', '|arbete..1|arbete..2|'],
       ['själv', 'JJ', '|själv..1|själv..2|själv..3|'],
       ['.', 'MAD', '|']],
      dtype='<U104')

In [3]:
print(line)

<w posset="|PAD|" pos="PAD" msd="PAD" lemma="|" lex="|" saldo="|" prefix="|" suffix="|" ref="1" dephead="2" deprel="IR">(</w>



In [139]:
import re
import numpy as np

In [158]:
def parse(line):
    word = re.search('(?<=>)(.*)(?=<)', line).group(0)
    pos = re.search('(?<=pos=)(.*\s)(?=m)', line).group(0).strip(' ').strip('"')
    saldo = re.search('(?<=saldo=)(.*\s)(?=p)', line).group(0).strip(' ').strip('"')
    #print (word, '\n', pos, '\n', saldo, '\n')
    return np.array([word,pos,saldo])

In [176]:
#voc = np.zeros((1,3))
#print (np.shape(voc))

line = '<w posset="|PAD|" pos="PAD" msd="PAD" lemma="|" lex="|" saldo="|" prefix="|" suffix="|" ref="1" dephead="2" deprel="IR">groda</w>'

line2 = 'hej då'

x = parse(line)
y = parse(line)

x

array(['groda', 'PAD', '|'],
      dtype='<U5')

In [177]:
voc=np.vstack([x,y])

In [179]:
voc[0]

array(['groda', 'PAD', '|'],
      dtype='<U5')